# Alison's approach to regression

## CRoss-Industry Standard Process for Data Mining (CRISP-DM)

Before we dig into the problem, lets refresh our memories on the steps in the CRISP-DM model.

<img src="img/new_crisp-dm.png" width="500">

### The Data

<img src="img/grocery-cart.jpg" width="500">

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities.

The data is located in the csv called `big_mart.csv`

### Step 1: Business Understanding

We previously explored the features of the big_mart dataset but now BigMart wants us to answer the following question:

**The sales team at BigMart HQ wants to know how to improve sales at their stores. They have sales data from different BigMart stores that records both item and store information. They have asked you to build a model to help predict which products at which stores sell well using the BigMart dataset.**


### Step 2: Data Understanding

We have already done a great deal of exploratory data analysis of this dataset.  Let's refresh out memory on what the original data contained. 

<img src="img/big_mart_data_variables.png" width="500">

### Step 3: Data Preparation

This step has already been done for you.  The following steps were taken and are reflected in the `big_mart_clean.csv` file.

- Imputed missing values for `Outlet_Size` (replaced missing with the mode)
- Imputed missing values for `Item_Weight` (replaced missing with the average)
- Cleaned typos in `Item_Fat_Content`
- Created new variable `Item_Type_Combined` (labels items as food, non-consumable, or drinks)
- Created new variable `Outlet_Years` (the years of operation of a store)


#### Before we begin modeling let's do a quick exploration of out new dataset!

In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures


import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels
import scipy

%matplotlib inline


big_mart = pd.read_csv('big_mart_clean.csv')
big_mart_orig = pd.read_csv('big_mart.csv')
big_mart.drop(columns=['Item_Sales_Cat'])
big_mart_reg = pd.concat([big_mart.drop(columns=['Item_Sales_Cat']), big_mart_orig['Item_Outlet_Sales']], axis=1)

In [ ]:
#set random state for our notebook
import numpy as np
np.random.seed(217)

y = big_mart_reg['Item_Outlet_Sales']
X = big_mart_reg.drop(columns=['Item_Outlet_Sales', 'Item_Identifier'])

#split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)

In [ ]:
# Preprocessing

numeric_features = ['Item_Weight', 'Item_Visibility',
                     'Item_MRP', 'Outlet_Years']
ss=StandardScaler()

categorical_features = [ 'Item_Fat_Content',
       'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Type_Combined' ]

categorical_transformer = Pipeline(steps=[
    ("ordinal", OrdinalEncoder()),
    ('onehot', OneHotEncoder(categories='auto',drop='first'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', ss, numeric_features),
        ('cat', categorical_transformer, categorical_features)], remainder='passthrough')

#let's use it now!

X_train_tran = preprocessor.fit_transform(X_train)
X_train_tran

#and we can transform X_test too

X_test_tran = preprocessor.transform(X_test)
X_test_tran

#get column names for categories
cat_names = preprocessor.named_transformers_['cat'].named_steps['ordinal'].categories_

cat_names = [val for sublist in cat_names for val in sublist[1:]]
cat_names

#full list of column names
column_names = numeric_features + cat_names

#apply column names to dataframe
X_train_trans = pd.DataFrame(X_train_tran.toarray(), columns=column_names)
X_train_trans.head()

### Step 4: Modeling

Once we have clean data, we can begin modeling! Remember, modeling, as with any of these other steps, is an iterative process. During this stage, we'll try to build and tune models to get the highest performance possible on our task.

Consider the following questions during the modeling step:

- Is this a classification task? A regression task? Something else?
- What models will we try?
- How do we deal with overfitting?
- Do we need to use regularization or not?
- What sort of validation strategy will we be using to check that our model works well on unseen data?
- What loss functions will we use?
- What threshold of performance do we consider as successful?



#### Dummy Model

First we are going to start with a dummy model to predict if the product has high or low sales. In our dummy model we classify everything as the majority class.
https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html

In [ ]:
# DummyClassifier to predict only target 0
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train_tran, y_train)
y_hat_train = dummy.predict(X_train_tran)
r2_score(y_hat_train, y_train)

#### Two approaches
##### Sklearn vs statsmodels

In [ ]:
#your code here
lin_mod1 = LinearRegression()

lin_mod1.fit(X_train_trans,y_train)
lin_mod1.score(X_train_trans,y_train)

In [ ]:
X_train_trans.Outlet_Years.hist()

In [ ]:
pd.DataFrame(data=zip(X_train_trans.columns,lin_mod1.coef_), columns = ['variables', 'coefficients'])

In [ ]:
X_train.Outlet_Type.value_counts()

In [ ]:
sns

In [ ]:
more_terms=PolynomialFeatures(interaction_only=True)

In [ ]:
more_terms.get_feature_names(X_train_trans.columns)

In [ ]:
more_vars = more_terms.fit_transform(X_train_trans)

In [ ]:
more_vars.shape

In [ ]:
lin_mod2 = LinearRegression()

In [ ]:
lin_mod2.fit(more_vars,y_train )

In [ ]:
lin_mod2.score(more_vars,y_train)

### Step 5: Evalution

During the evaluation step we want to evaluate the results of our models, and decide the next steps in selecting the "best" model.  During this step we should consider the following:

- Does our model solve the business problem?
- What metrics should we be using to evaluate the "success" of our model?
- Can we further improve our models?
- Do we need more data?  Or different data?

In [ ]:
# some helpful code for me later on

sns.pairplot(df)b
plt.show()
fig = sm.qqplot(normal_rv, line = 'r')
plt.show()